In [10]:
##set up
import numpy as np
import pandas as pd
import string
import email
import os ##use this to traverse folders
from email.parser import Parser
import re
import lxml.html
import collections
import pickle

In [11]:
#extract labels from TREC2007/full/ file lists
with open('trec07p/full/index', 'r') as ifile:
    raw_labels = ifile.readlines()

#using file list to give labels and e-mail numbers

categories = [i.split( )[0] for i in raw_labels]
email_nums = [i.split(".")[-1][:-1] for i in raw_labels]
labels = pd.Series(categories, index=email_nums)

In [12]:
# extract the body of email
def walkMsg(msg):
    """
    Input: string of email
    Output: body of email
    """
    for part in msg.walk():
        if part.get_content_type() == "multipart/alternative":
            continue
        yield part.get_payload(decode=1)

In [13]:
##encode
def robust_decode(bs):
    '''Takes a byte string as param and convert it into a unicode one.
First tries UTF8, and fallback to Latin1 if it fails'''
    cr = None
    try:
        #cr = bs.decode('utf8')
        cr = bs.decode('utf-8-sig')
    except UnicodeDecodeError:
        cr = bs.decode('ISO-8859-1')
        cr = cr.encode("ascii", "ignore")
        cr = cr.decode('utf8')
    return cr

In [14]:
def info_of_email_2():
    """
    Input: path, list of files name, classification(spam or ham)
    Ourput: a list with info of email
    """
    
    parser = Parser()
    info = []
    for nums in range(len(categories)):
        filepath = 'trec07p/data/inmail.%s' % email_nums[nums]
        
        with open(filepath, 'r') as ifile:
            email = parser.parse(ifile)
            
            Date = email.get("Date")

            Content = " ".join([robust_decode(i) for i in walkMsg(email) if i != None])
            Content = ' '.join(Content.splitlines())
            Content = Content.replace("\t","")
            
            if 'html' in Content.lower():
                Content = lxml.html.fromstring(Content).text_content()
            try: 
                year = re.search('\d{4}',Date).group(0)
            except:
                year = None
            try:
                hour = re.search('\d{2}:\d{2}:\d{2}',Date).group(0).split(":")[0]
            except:
                hour = None
            try: 
                weekday = re.search('Mon|Tue|Wed|Thu|Fri|Sat|Sun',Date).group(0)
            except: 
                weekday =None
            try:
                month = re.search('Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec|Sept',Date).group(0)
            except:
                month =None
            info.append([year, weekday, hour, month,
                         email.get("From"),email.get('To'),
                         email.get('cc'),email.get('bcc'), 
                         str(email.get("Content-Type")).split(";")[0].lower(),
                         email.get('subject'),
                         Content,
                         categories[nums]])
    return info

In [15]:
trec07 = pd.DataFrame(info_of_email_2(),
                 columns=['Year','Weekday',"Hour","Month",
                          'From','To','cc','bcc','Content_type','Subjects','Content','Classification'])

In [16]:
##Final Date
trec07 = trec07.loc[trec07.Year.isin(['1999','2000','2001','2002','2003','2004','2005','2006','2007'])]

In [ ]:
import pickle
trec07.to_pickle("trec07_email.txt")
trec07_pickle = pickle.load(open("trec07_email.txt","rb"))

In [9]:
#Text preprocessing
#tokenizing, filtering of stopwords
#word dictionary of features
#transform documents to feature vectors:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(trec07['Content'])


In [1]:
#X_train_counts.get_feature_names()

NameError: name 'X_train_counts' is not defined

In [22]:
X_train_tf[:100,:100].todense()

matrix([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        ..., 
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.]])